In [1]:
import os
import numpy as np
import fiona
import pandas as pd
import geopandas as gpd
import argparse
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
from mpl_toolkits.axes_grid1.anchored_artists import AnchoredSizeBar
import matplotlib.font_manager as fm
from textwrap import wrap
from matplotlib.backends.backend_pdf import PdfPages


def valid_path(arg):
    arg = os.path.abspath(arg)
    if not os.path.exists(arg):
        msg = f"The path {arg} does not exist!"
        raise argparse.ArgumentTypeError(msg)
    else:
        return arg


In [2]:
# Parse input arguments
# parser = argparse.ArgumentParser(description='Generate origin destination matrix')
# parser.add_argument('-gpkg_cities',
#                     help='path to all cities summary results geopackage',
#                     default='./data/output/November 2020/global_indicators_city_2020-11-24.gpkg',
#                     type=valid_path)
# parser.add_argument('-gpkg_hexes',
#                     help='path to all cities hexagon grid results geopackage',
#                     default='./data/output/November 2020/global_indicators_hex_250m_2020-11-24.gpkg',
#                     type=valid_path)
# args = parser.parse_args()

# dummy parsing for interactive debugging
class Object(object):
    pass

args = Object()
args.gpkg_cities = os.path.abspath('../process/data/output/global_indicators_city_2020-11-24.gpkg')
args.gpkg_hexes = os.path.abspath('../process/data/output/global_indicators_hex_250m_2020-11-24.gpkg')

cities = gpd.read_file(args.gpkg_cities, layer='all_cities_combined')

# cities

In [3]:
hexes={}
for city in cities.City:
    hexes[city] = gpd.read_file(args.gpkg_hexes, layer=city.lower().replace(' ','_'))

In [4]:
hexes.keys()

dict_keys(['Maiduguri', 'Mexico City', 'Baltimore', 'Phoenix', 'Seattle', 'Sao Paulo', 'Hong Kong', 'Chennai', 'Bangkok', 'Hanoi', 'Adelaide', 'Melbourne', 'Sydney', 'Auckland', 'Graz', 'Ghent', 'Olomouc', 'Odense', 'Cologne', 'Lisbon', 'Barcelona', 'Valencia', 'Vic', 'Bern', 'Belfast'])

In [5]:
print("""
Global Indicators project, 2020

Analysis and mapping of threshold values for urban design and transport planning features associated with
  - (A) ≥80% probability of engaging in walking for transport and 
  - (B) reaching the WHO’s target of a ≥15% relative reduction in insufficient physical activity through walking

""")





Global Indicators project, 2020

Analysis and mapping of threshold values for urban design and transport planning features associated with
  - (A) ≥80% probability of engaging in walking for transport and 
  - (B) reaching the WHO’s target of a ≥15% relative reduction in insufficient physical activity through walking




In [48]:
# Analysis set up
scenarios={
  'A':'≥80% probability of engaging in walking for transport', 
  'B':'reaching the WHO’s target of a ≥15% relative reduction in insufficient physical activity through walking',
  'distances':'distances to destinations, measured up to a maximum distance target threshold of 500 metres'
}
greq = '≥'
thresholds = {
'Mean 1000 m neighbourhood population per km²':{
  'data':'hexes',
  'variable':'local_nh_population_density',
  'scenarios':{
      'A':{
        'threshold':5665,
        'interval':(4790, 6750),
        'interval_type':'95% CrI'
        },
      'B':{
        'threshold':6491,
        'interval':(5677, 7823),
        'interval_type':'95% CrI' 
        }
  }
},
'Mean 1000 m neighbourhood street intersections per km²':{
  'data':'hexes',
  'variable':'local_nh_intersection_density',
  'scenarios':{
      'A':{
        'threshold':98,
        'interval':(90, 110),
        'interval_type':'95% CrI'
        },
      'B':{
        'threshold':122,
        'interval':(106, 156),
        'interval_type':'95% CrI'
        }
  }
},
# 'Public transport density (stops/km2)':{
#   'data':'points',
#   'variable':'',
#   'A':{
#     'threshold':28,
#     'interval':(23, max),
#     'interval_type':'95% CrI'
#     },
#   'B':{
#     'notes':'Not a significant correlate'
#     }
# },
'Distance to nearest public transport stops (m; up to 500m)':{
  'data':'points',
  'variable':'pct_access_500m_pt_any_binary',
  'scenarios':{
      'distances':{
        'interval':(0,150,200,300,400,500),
        'interval_type':'distance (m)',
        }
  }
},
'Distance to nearest park (m; up to 500m)':{
  'data':'points',
  'variable':'pct_access_500m_public_open_space_any_binary',
  'scenarios':{
      'distances':{
        'interval':(0,150,200,300,400,500),
        'interval_type':'distance (m)',
        }
  }
}}



In [52]:
fontprops = fm.FontProperties(size=8)
#for city in hexes.keys():
for city in ['Maiduguri','Ghent']:
    print(city)
    study_region = cities.query(f'City=="{city}"').to_crs(hexes[city].crs).copy()
    # create a PdfPages object
    with PdfPages(f'reports/{city}_threshold_summary.pdf') as pdf:
        for indicator in thresholds.keys():
            data = thresholds[indicator]['data']
            variable = thresholds[indicator]['variable']
            indicator_scenarios = list(thresholds[indicator]['scenarios'].keys())
            if data == 'hexes':
                var_min = min(hexes[city][variable])        
                var_max = max(hexes[city][variable])
                # limits = (min(thresholds[indicator]['A']['interval'][0],thresholds[indicator]['B']['interval'][0],var_min),
                #          max(thresholds[indicator]['A']['interval'][0],thresholds[indicator]['B']['interval'][0],var_max))

                # map main indicator
                fig, ax = plt.subplots(1, 1, figsize=(11.69,8.27))
                ax.set_aspect('equal')
                study_region.plot(ax=ax, color='none', edgecolor='black',zorder=2)
                divider = make_axes_locatable(ax)
                cax = divider.append_axes("right", size="5%", pad=0.1)
                ax.set_xticks([])
                ax.set_yticks([])

                scalebar = AnchoredSizeBar(ax.transData,
                                           1000, '1000 m', 'lower right', 
                                           pad= .01,
                                           color='black',
                                           frameon=False,
                                           fontproperties=fontprops)

                ax.add_artist(scalebar)
                fig.suptitle("\n".join(wrap(indicator, 120 )))
                hexes[city].plot(column=variable, ax=ax, legend=True, cax=cax, cmap='viridis_r', zorder=1)
                pdf.savefig(fig)
                plt.clf()

                # map scenarios using custom splits
                interval_splits ={}
                splits = {}
                for scenario in [s for s in scenarios.keys() if s in indicator_scenarios]:
                    attributes = list(thresholds[indicator]['scenarios'][scenario].keys())
                    if ('interval' in attributes):
                        splits[scenario] = thresholds[indicator]['scenarios'][scenario]['interval']
                        interval_type = thresholds[indicator]['scenarios'][scenario]['interval_type']
                        if max in splits:
                            splits[scenario] = [x if x!=max else var_max for x in splits[scenario]]
                        if min in splits:
                            splits[scenario] = [x if x!=min else min(hexes[city][variable]) for x in splits[scenario]]
                        interval_splits[scenario] = list(splits[scenario]).copy()
                        split_labels = [f'within {interval_type} {splits[scenario]}']
                        if var_min < splits[scenario][0]:
                            splits[scenario] = [var_min]+list(splits[scenario])
                            split_labels = [f'below {interval_type} lower bound']+split_labels
                        if var_max > splits[scenario][-1]:
                            splits[scenario] = list(splits[scenario])+[var_max]
                            split_labels = split_labels+[f'exceeds {interval_type} upper bound']
                        #print(splits)
                        hexes[city][f'{variable}_{scenario}'] = pd.cut(hexes[city][variable], bins=splits[scenario], labels=split_labels)
                        hexes[city][f'{variable}_{scenario}']
                        fig, ax = plt.subplots(figsize=(11.69,8.27))
                        ax.set_aspect('equal')
                        study_region.plot(ax=ax, color='none', edgecolor='black', zorder=2)
                        ax.set_xticks([])
                        ax.set_yticks([])
                        scalebar = AnchoredSizeBar(ax.transData,
                                                   1000, '1000 m', 'lower right', 
                                                   pad= .01,
                                                   color='black',
                                                   frameon=False,
                                                   fontproperties=fontprops)
                        ax.add_artist(scalebar)
                        fig.suptitle("\n".join(wrap(f'Estimated {indicator} requirement for {scenarios[scenario]}', 120 )))
                        if 'notes' in attributes:
                            ax.set_title(f"{thresholds[indicator]['scenarios'][scenario]['notes']}")
                        
                        hexes[city].plot(column = f'{variable}_{scenario}',ax=ax,legend=True,cmap='viridis_r', zorder=1)
                        pdf.savefig(fig)
                        plt.clf()
                
                hexes[city][f'{variable}'].hist(grid=False)
                plt.suptitle("\n".join(wrap(f'Histogram of {indicator}.',120)))
                
                subtitle = ''
                scenario_style = {'A':{'colour':'lightcyan','line':'dashed'},'B':{'colour':'palegreen','line':'solid'}}
                for scenario in [s for s in scenarios.keys() if s in indicator_scenarios]:
                    attributes = list(thresholds[indicator]['scenarios'][scenario].keys())
                    if ('interval' in attributes):
                        subtitle = subtitle+(f'Target threshold of {thresholds[indicator]["scenarios"][scenario]["threshold"]} '
                                             f'({interval_type} {thresholds[indicator]["scenarios"][scenario]["interval"]};{scenario_style[scenario]["colour"]}, {scenario_style[scenario]["line"]}).')
                        for line in splits[scenario][1:]:
                            plt.axvline(line, color='k', linestyle=scenario_style[scenario]["line"], linewidth=1)
                        plt.axvspan(*interval_splits[scenario], color=scenario_style[scenario]["colour"],alpha=0.6, zorder=0)
                if subtitle!='':
                    plt.title("\n".join(wrap(subtitle,120)))
                
                pdf.savefig(fig)
                plt.clf()


Maiduguri
Ghent


<Figure size 841.68x595.44 with 0 Axes>

<Figure size 841.68x595.44 with 0 Axes>

<Figure size 841.68x595.44 with 0 Axes>

<Figure size 841.68x595.44 with 0 Axes>

...

<Figure size 841.68x595.44 with 0 Axes>

<Figure size 841.68x595.44 with 0 Axes>

<Figure size 841.68x595.44 with 0 Axes>

<Figure size 841.68x595.44 with 0 Axes>

<Figure size 841.68x595.44 with 0 Axes>

In [30]:
hexes[city][f'{variable}'].describe()

count    2176.000000
mean       60.551672
std        43.516081
min         0.000000
25%         0.000000
50%        87.878788
75%       100.000000
max       100.000000
Name: pct_access_500m_public_open_space_any_binary, dtype: float64

In [35]:
list(thresholds[indicator].keys())

['variable', 'A', 'B']

In [61]:
bounds = cities.query('City=="Maiduguri"').to_crs(hexes['Maiduguri'].crs).bounds
width = bounds['maxx']-bounds['minx']
height = bounds['maxy']-bounds['miny']
ax.set(xlim=(bounds['minx']+width/2,bounds['maxx']), ylim=(0,60))
bound


,minx,miny,maxx,maxy
0,289006.751226,1.303221e+06,307867.780447,1.314103e+06


In [ ]:
ax.set(xlim=(-140,-50), ylim=(0,60))